# Filter Comparison

This notebook compares median, Gaussian, Non-Local Means (NLM), and anisotropic diffusion filtering on a random subset of images from the metadata CSV.

## Setup
Load dependencies and define the path to the metadata CSV.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, restoration
from scipy.ndimage import median_filter, gaussian_filter

# Path to your metadata CSV (expects a `filepath` column)
metadata_csv = 'data/metadata.csv'

# Read dataframe
df = pd.read_csv(metadata_csv)
print(f'Loaded {len(df)} samples')

## Select example images
Randomly sample 10 images from the CSV for filtering.

In [ ]:
sample_df = df.sample(n=10, random_state=42).reset_index(drop=True)
image_paths = sample_df['filepath'].tolist()
print('Selected images:')
for p in image_paths:
    print(p)

## Apply filters
For each image we apply median, Gaussian, NLM and anisotropic diffusion filtering and visualize the results.

In [ ]:
results = []
for path in image_paths:
    img = io.imread(path)
    if img.ndim > 2:
        img = img[..., 0]
    img = img.astype(float) / 255.0

    median_img = median_filter(img, size=3)
    gaussian_img = gaussian_filter(img, sigma=1)
    nlm_img = restoration.denoise_nl_means(img, fast_mode=True, patch_size=5, patch_distance=3, h=0.1)
    aniso_img = restoration.denoise_tv_chambolle(img, weight=0.1)

    results.append((img, median_img, gaussian_img, nlm_img, aniso_img))

In [ ]:
for i, (orig, med, gauss, nlm, aniso) in enumerate(results):
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    titles = ['Original', 'Median', 'Gaussian', 'NLM', 'Anisotropic']
    for ax, im, t in zip(axes, [orig, med, gauss, nlm, aniso], titles):
        ax.imshow(im, cmap='gray')
        ax.set_title(t)
        ax.axis('off')
    plt.show()